不動産価格予測

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install h2o >> /dev/null

In [3]:
!pip install optuna xfeat japanize-matplotlib >> /dev/null

In [4]:
# パスの設定
My_PATH = "/content/drive/MyDrive/property_values_prediction"

%cd {My_PATH}

/content/drive/MyDrive/property_values_prediction


In [5]:
# zipファイルの解凍

# 古いdata.zipの削除
!rm -r /content/drive/MyDrive/property_values_prediction/data

import zipfile
with zipfile.ZipFile('data.zip', 'r')as f:
    f.extractall('./data')

In [6]:
# train.zipの解凍
# 古いtrain.zipの削除
!rm -r /content/drive/MyDrive/property_values_prediction/data/train

with zipfile.ZipFile('data/train.zip', 'r')as f:
    f.extractall('./data/')

!rm -r /content/drive/MyDrive/property_values_prediction/data/__MACOSX

rm: cannot remove '/content/drive/MyDrive/property_values_prediction/data/train': No such file or directory


In [7]:
import datetime
import os

now = datetime.datetime.now()
current_time = now.strftime("%Y-%m-%d-%H-%M")
dir_for_output = "./output/" + current_time

os.makedirs(dir_for_output, exist_ok=True)

In [8]:
import re

import japanize_matplotlib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
import seaborn as sns
sns.set(font="IPAexGothic")

from glob import glob
from functools import partial
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from xfeat import (
    SelectCategorical,
    LabelEncoder,
    LambdaEncoder,
    Pipeline,
    ConcatCombination,
    SelectNumerical,
    ArithmeticCombinations,
    TargetEncoder,
    aggregation,
    GBDTFeatureSelector,
    GBDTFeatureExplorer,
)

h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes
)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpopn_qr96
  JVM stdout: /tmp/tmpopn_qr96/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpopn_qr96/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,1 month and 22 days
H2O_cluster_name:,H2O_from_python_unknownUser_ymxh28
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [9]:
def normalize_moyori(moyori):
    if moyori == moyori:
        if moyori == '30分?60分':
            moyori = 45
        elif moyori == '1H?1H30':
            moyori = 75
        elif moyori == '1H30?2H':
            moyori = 105
        elif moyori == '2H?':
            moyori = 120
        moyori = int(moyori)
    return moyori

def normalize_area(area):
    if area == area:
        area = int(re.sub('m\^2未満|㎡以上', '', str(area)))
    return area

def convert_wareki_to_seireki(wareki):
    if wareki == wareki:
        if wareki == '戦前':
            wareki = '昭和20年'
        value = wareki[2:-1]
        if value == '元':
            value = 1
        else:
            value = int(value)
        if '昭和' in wareki:
            seireki = 1925+value
        elif '平成' in wareki:
            seireki = 1988+value
        elif '令和' in wareki:
            seireki = 2018+value
    else:
        seireki = wareki
    return seireki

コンペデータ作成（利用カラム、目的変数など決める）

dataディレクトリ配下に以下のようにデータを格納しているとする

├─ input<br>
│   ├─ data_explanation.xlsx<br>
│   ├─ sample_submission.csv<br>
│   ├─ test.csv<br>
│   └─ train<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├─ 01.csv<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├─ 02.csv<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├─ 03.csv<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├─ ...<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└─ 47.csv<br>
└─ output


In [10]:
BASE_PATH = './data/'
RANDOM_STATE = 10

In [11]:
paths = glob(BASE_PATH + 'train/*')
train_dfs = []
for path in paths:
    train_df = pd.read_csv(path)
    train_dfs.append(train_df)
train_df = pd.concat(train_dfs)
train_df.reset_index(drop=True, inplace=True)
test_df = pd.read_csv(BASE_PATH + 'test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.



In [12]:
train_df.isnull().sum()

ID                   0
種類                   0
地域              677392
市区町村コード              0
都道府県名                0
市区町村名                0
地区名                660
最寄駅：名称            2672
最寄駅：距離（分）        23098
間取り              23661
面積（㎡）                0
土地の形状           677392
間口              677392
延床面積（㎡）         677392
建築年              19622
建物の構造            16486
用途               58246
今後の利用目的         364049
前面道路：方位         677392
前面道路：種類         677392
前面道路：幅員（ｍ）      677392
都市計画             19221
建ぺい率（％）          23345
容積率（％）           23345
取引時点                 0
改装               61966
取引の事情等          658854
取引価格（総額）_log         0
dtype: int64

In [13]:
sub_df = pd.read_csv(BASE_PATH + 'sample_submission.csv')

In [14]:
ID = 'ID'
TARGET = '取引価格（総額）_log'
rm_cols = []

In [15]:
df = pd.concat([train_df, test_df])

rm_cols += ['市区町村コード']
for i,v in df.nunique().iteritems():
    if v <= 1:
        rm_cols.append(i)

rm_cols

['市区町村コード',
 '種類',
 '地域',
 '土地の形状',
 '間口',
 '延床面積（㎡）',
 '前面道路：方位',
 '前面道路：種類',
 '前面道路：幅員（ｍ）']

In [16]:
test_df[TARGET] = np.nan
train_df.drop(rm_cols, axis=1, inplace=True)
test_df.drop(rm_cols, axis=1, inplace=True)
df = pd.concat([train_df, test_df])
df.sort_values('取引時点', inplace=True)
df.reset_index(drop=True, inplace=True)
df.shape

(700543, 19)

In [17]:
df['取引時点'].unique()

array(['2005年第３四半期', '2005年第４四半期', '2006年第１四半期', '2006年第２四半期',
       '2006年第３四半期', '2006年第４四半期', '2007年第１四半期', '2007年第２四半期',
       '2007年第３四半期', '2007年第４四半期', '2008年第１四半期', '2008年第２四半期',
       '2008年第３四半期', '2008年第４四半期', '2009年第１四半期', '2009年第２四半期',
       '2009年第３四半期', '2009年第４四半期', '2010年第１四半期', '2010年第２四半期',
       '2010年第３四半期', '2010年第４四半期', '2011年第１四半期', '2011年第２四半期',
       '2011年第３四半期', '2011年第４四半期', '2012年第１四半期', '2012年第２四半期',
       '2012年第３四半期', '2012年第４四半期', '2013年第１四半期', '2013年第２四半期',
       '2013年第３四半期', '2013年第４四半期', '2014年第１四半期', '2014年第２四半期',
       '2014年第３四半期', '2014年第４四半期', '2015年第１四半期', '2015年第２四半期',
       '2015年第３四半期', '2015年第４四半期', '2016年第１四半期', '2016年第２四半期',
       '2016年第３四半期', '2016年第４四半期', '2017年第１四半期', '2017年第２四半期',
       '2017年第３四半期', '2017年第４四半期', '2018年第１四半期', '2018年第２四半期',
       '2018年第３四半期', '2018年第４四半期', '2019年第１四半期', '2019年第２四半期',
       '2019年第３四半期', '2019年第４四半期', '2020年第１四半期', '2020年第２四半期',
       '2020年第３四半期', '2020年第４四半期', '2021年第１四半期'], dtype

In [18]:
val_min_idx = min(df[df['取引時点'].str.contains('2020年第２四半期|2020年第３四半期', regex=True)].index)
test_min_idx = min(df[df['取引時点'].str.contains('2020年第４四半期|2021年第１四半期', regex=True)].index)
val_min_idx, test_min_idx

(652493, 677392)

In [19]:
set(df.iloc[val_min_idx:test_min_idx, :]['取引時点'].values)

{'2020年第２四半期', '2020年第３四半期'}

特徴量生成

In [20]:
enc_dic = {}
for i, e in enumerate(sorted(list(set(df['取引時点'].values)))):
    enc_dic[e] = i
df['取引時点_enc'] = df['取引時点'].map(enc_dic)

In [21]:
te_dic = {}
time_col = '取引時点_enc'
group_col = '都道府県名'

for i in set(df[time_col].values):
    tmp_df = df[df[time_col] < i]
    te_dic[i] = tmp_df.groupby(group_col)[TARGET].agg('mean').to_dict()

te_dic[50]

{'三重県': 7.099989772836443,
 '京都府': 7.15593584855535,
 '佐賀県': 7.009937258263499,
 '兵庫県': 7.157742870483325,
 '北海道': 6.965967793677889,
 '千葉県': 7.155825324965048,
 '和歌山県': 7.0226902108157585,
 '埼玉県': 7.169744271396765,
 '大分県': 6.911537826573121,
 '大阪府': 7.158352973821433,
 '奈良県': 7.0568846102346985,
 '宮城県': 7.03954425380504,
 '宮崎県': 6.958678653711221,
 '富山県': 7.046920071386999,
 '山口県': 7.045342730349815,
 '山形県': 7.087990602814885,
 '山梨県': 6.848839888686703,
 '岐阜県': 7.078095913486438,
 '岡山県': 7.038090734927505,
 '岩手県': 6.92500267906058,
 '島根県': 7.16537621470031,
 '広島県': 7.09344902165506,
 '徳島県': 6.912051981629739,
 '愛媛県': 6.9664330115966635,
 '愛知県': 7.107301969238665,
 '新潟県': 6.913835156235099,
 '東京都': 7.377929204637786,
 '栃木県': 6.950636958071022,
 '沖縄県': 7.160046604428345,
 '滋賀県': 7.166026958168428,
 '熊本県': 6.970182251843288,
 '石川県': 6.929363348877184,
 '神奈川県': 7.258118267209654,
 '福井県': 6.979575815840192,
 '福岡県': 6.9922649050422665,
 '福島県': 6.974380579140183,
 '秋田県': 6.94131374711262,
 

In [22]:
def calc_te(row):
    if row[time_col] in te_dic and row[group_col] in te_dic[row[time_col]]:
        return te_dic[row[time_col]][row[group_col]]
    else:
        return 0

df[group_col+'_te'] = df.apply(calc_te, axis=1)
df.head(2)

,ID,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時点_enc,都道府県名_te
0,27232131,大阪府,大阪市旭区,新森,森小路,6,１ＬＤＫ,50,昭和49年,ＲＣ,住宅,NaN,準工業地域,80.0,200.0,2005年第３四半期,改装済,NaN,7.041393,0,0.0
1,14289301,神奈川県,川崎市高津区,千年新町,武蔵新城,7,３ＬＤＫ,60,平成4年,ＲＣ,住宅,NaN,第１種中高層住居専用地域,60.0,200.0,2005年第３四半期,改装済,NaN,7.361728,0,0.0


In [23]:
df['取引時点_何年前'] = df['取引時点'].apply(lambda x: 2021-int(x[:4]))
df.drop(['取引時点'], axis=1, inplace=True)
df['建築年'] = df['建築年'].apply(lambda x: convert_wareki_to_seireki(x))
df['面積（㎡）'] = df['面積（㎡）'].apply(lambda x: normalize_area(x))
df['最寄駅：距離（分）'] = df['最寄駅：距離（分）'].apply(lambda x: normalize_moyori(x))

In [24]:
num_df = SelectNumerical().fit_transform(df)
num_df.head(2)

,ID,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名_te,取引時点_何年前
0,27232131,6.0,50,1974.0,80.0,200.0,7.041393,0,0.0,16
1,14289301,7.0,60,1992.0,60.0,200.0,7.361728,0,0.0,16


In [25]:
encoder = Pipeline([
    SelectCategorical(),
    LabelEncoder(output_suffix=""),
])

le_df = encoder.fit_transform(df)
le_df.head(2)

,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等
0,0,0,0,0,0,0,0,-1,0,0,-1
1,1,1,1,1,1,0,0,-1,1,0,-1


In [26]:
encoder = Pipeline(
    [
        SelectNumerical(),
        ArithmeticCombinations(
            input_cols=["面積（㎡）", "容積率（％）"], 
            drop_origin=True, 
            operator="*", 
            r=2,
        ),
    ]
)

num_comb_df = encoder.fit_transform(df)/100
num_comb_df.head(2)

,面積（㎡）容積率（％）_combi
0,100.0
1,120.0


In [27]:
agg_dfs = []

def get_agg_df(df, group_col):

    agg_df, agg_cols = aggregation(df,
                        group_key=group_col,
                        group_values=['最寄駅：距離（分）', '面積（㎡）', '建ぺい率（％）', '容積率（％）'],
                        agg_methods=['count', 'mean', 'min', 'max'],
                        )

    return agg_df[agg_cols]

group_col = '市区町村名'
agg_dfs.append(get_agg_df(df, group_col))
agg_dfs[0].head(2)

,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名
0,789,808,806,806,8.206591,59.814356,70.397022,236.228288,0.0,15,60.0,200.0,23.0,115,80.0,400.0
1,2945,3089,2872,2872,13.020713,59.043380,61.922006,212.580084,0.0,15,40.0,80.0,105.0,550,80.0,1100.0


In [28]:
feat_df = pd.concat([num_df,le_df,num_comb_df]+agg_dfs, axis=1)
print(feat_df.shape)

(700543, 38)


In [29]:
feat_df.dtypes

ID                                   int64
最寄駅：距離（分）                          float64
面積（㎡）                                int64
建築年                                float64
建ぺい率（％）                            float64
容積率（％）                             float64
取引価格（総額）_log                       float64
取引時点_enc                             int64
都道府県名_te                           float64
取引時点_何年前                             int64
都道府県名                                int64
市区町村名                                int64
地区名                                  int64
最寄駅：名称                               int64
間取り                                  int64
建物の構造                                int64
用途                                   int64
今後の利用目的                              int64
都市計画                                 int64
改装                                   int64
取引の事情等                               int64
面積（㎡）容積率（％）_combi                  float64
agg_count_最寄駅：距離（分）_grpby_市区町村名      int64
agg_count_面

モデル構築

In [30]:
train_df = feat_df.iloc[:test_min_idx, :]
test_df = feat_df.iloc[test_min_idx:, :]
print(train_df.shape, test_df.shape)

(677392, 38) (23151, 38)


In [31]:
train_df.head()

,ID,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名_te,取引時点_何年前,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等,面積（㎡）容積率（％）_combi,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名
0,27232131,6.0,50,1974.0,80.0,200.0,7.041393,0,0.0,16,0,0,0,0,0,0,0,-1,0,0,-1,100.0,789,808,806,806,8.206591,59.814356,70.397022,236.228288,0.0,15,60.0,200.0,23.0,115,80.0,400.0
1,14289301,7.0,60,1992.0,60.0,200.0,7.361728,0,0.0,16,1,1,1,1,1,0,0,-1,1,0,-1,120.0,2945,3089,2872,2872,13.020713,59.043380,61.922006,212.580084,0.0,15,40.0,80.0,105.0,550,80.0,1100.0
2,13417040,NaN,60,1993.0,60.0,200.0,7.255273,0,0.0,16,2,2,2,2,2,0,0,-1,0,0,-1,120.0,4605,4893,4812,4812,9.963084,50.796035,64.170823,279.640482,0.0,15,40.0,80.0,45.0,270,80.0,600.0
3,14267697,NaN,85,1982.0,NaN,NaN,7.556303,0,0.0,16,1,3,3,3,1,-1,0,-1,-1,-1,-1,NaN,3465,3675,3375,3375,13.894372,67.380952,59.454815,170.755556,0.0,15,30.0,60.0,120.0,210,80.0,500.0
4,13386603,5.0,20,2005.0,80.0,500.0,7.278754,0,0.0,16,2,4,4,4,3,0,0,-1,2,1,-1,100.0,8377,8625,8526,8526,8.184075,46.059710,66.272578,274.364297,0.0,10,30.0,80.0,75.0,220,80.0,600.0


In [32]:
test_df = test_df.fillna(0)
test_df.head()

,ID,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名_te,取引時点_何年前,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等,面積（㎡）容積率（％）_combi,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名
677392,14068534,11.0,60,1996.0,60.0,200.0,0.0,61,7.268224,1,1,251,5181,1999,2,1,-1,0,0,1,-1,120.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0
677393,14068533,24.0,70,1998.0,50.0,100.0,0.0,61,7.268224,1,1,251,5181,1999,1,0,0,0,9,0,-1,70.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0
677394,14068532,11.0,60,1996.0,60.0,200.0,0.0,61,7.268224,1,1,251,5181,1999,1,1,-1,0,0,1,-1,120.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0
677395,14068420,21.0,105,1969.0,60.0,200.0,0.0,61,7.268224,1,1,251,5018,1702,19,0,0,0,1,0,-1,210.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0
677396,14068219,21.0,65,1979.0,60.0,200.0,0.0,61,7.268224,1,1,251,4480,1544,1,0,-1,0,7,1,-1,130.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0


In [33]:
feat_cols = [col for col in train_df.columns if col not in rm_cols+[ID, TARGET]]

In [34]:
cat_cols = list(le_df.columns) + ['取引時点_enc']
cat_cols

['都道府県名',
 '市区町村名',
 '地区名',
 '最寄駅：名称',
 '間取り',
 '建物の構造',
 '用途',
 '今後の利用目的',
 '都市計画',
 '改装',
 '取引の事情等',
 '取引時点_enc']

In [35]:
train_x = train_df[feat_cols]
train_y = train_df[TARGET]
test_x = test_df[feat_cols]
test_y = test_df[TARGET]

In [36]:
train = pd.merge(train_x, train_y, left_index=True, right_index=True)
test = pd.merge(test_x, test_y, left_index=True, right_index=True)

In [37]:
test.head()

,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引時点_enc,都道府県名_te,取引時点_何年前,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等,面積（㎡）容積率（％）_combi,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名,取引価格（総額）_log
677392,11.0,60,1996.0,60.0,200.0,61,7.268224,1,1,251,5181,1999,2,1,-1,0,0,1,-1,120.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,0.0
677393,24.0,70,1998.0,50.0,100.0,61,7.268224,1,1,251,5181,1999,1,0,0,0,9,0,-1,70.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,0.0
677394,11.0,60,1996.0,60.0,200.0,61,7.268224,1,1,251,5181,1999,1,1,-1,0,0,1,-1,120.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,0.0
677395,21.0,105,1969.0,60.0,200.0,61,7.268224,1,1,251,5018,1702,19,0,0,0,1,0,-1,210.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,0.0
677396,21.0,65,1979.0,60.0,200.0,61,7.268224,1,1,251,4480,1544,1,0,-1,0,7,1,-1,130.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,0.0


In [38]:
h2o_train = h2o.H2OFrame(train)
h2o_test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [39]:
# 説明変数/目的変数のカラムを指定
kw = h2o.H2OFrame(list(h2o_train.columns))

feature_cols = kw.columns[:-1]
target_cols = "取引価格（総額）_log"

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [40]:
%%time
# max_runtime_secsは最大実行時間を指定
aml = H2OAutoML(
    max_runtime_secs=3000,
    max_models=None,  # no limit
    stopping_metric ='mae',
    sort_metric ='mae',
    seed = RANDOM_STATE,
    )

aml.train(
    y=target_cols,
    training_frame=h2o_train,
    )

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
CPU times: user 6min 18s, sys: 11.4 s, total: 6min 29s
Wall time: 50min 29s


In [41]:
#スコアの高いモデル順に並び変え
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mae,mean_residual_deviance,rmse,mse,rmsle
GBM_1_AutoML_1_20211129_120921,0.0841433,0.0185798,0.136308,0.0185798,0.0175412
XGBoost_1_AutoML_1_20211129_120921,0.0879565,0.0192915,0.138894,0.0192915,0.0177931
DRF_1_AutoML_1_20211129_120921,0.114971,0.0341858,0.184894,0.0341858,0.023649
StackedEnsemble_BestOfFamily_2_AutoML_1_20211129_120921,0.128345,0.0330172,0.181706,0.0330172,0.0230961
StackedEnsemble_AllModels_1_AutoML_1_20211129_120921,0.128345,0.0330173,0.181707,0.0330173,0.0230961
StackedEnsemble_BestOfFamily_1_AutoML_1_20211129_120921,0.128346,0.0330183,0.181709,0.0330183,0.0230965
GLM_1_AutoML_1_20211129_120921,0.145866,0.0456194,0.213587,0.0456194,0.0262476
GBM_2_AutoML_1_20211129_120921,0.163622,0.0507367,0.225248,0.0507367,0.0283463
GBM_3_AutoML_1_20211129_120921,0.164942,0.0508623,0.225527,0.0508623,0.0283851
GBM_4_AutoML_1_20211129_120921,0.184563,0.0611686,0.247323,0.0611686,0.0310401


In [42]:
# Get the top model of leaderboard
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_1_AutoML_1_20211129_120921


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,51.0,51.0,1330837.0,15.0,15.0,15.0,620.0,3263.0,2073.745




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.015837973638433346
RMSE: 0.12584901127316553
MAE: 0.07728259363304413
RMSLE: 0.016233290421731227
Mean Residual Deviance: 0.015837973638433346

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 0.018579803930395825
RMSE: 0.13630775447639001
MAE: 0.0841433134975746
RMSLE: 0.01754123360437044
Mean Residual Deviance: 0.018579803930395825

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.084143,0.002014,0.084986,0.084884,0.084915,0.085374,0.080557
1,mean_residual_deviance,0.018580,0.000464,0.018565,0.018564,0.018638,0.019221,0.017911
2,mse,0.018580,0.000464,0.018565,0.018564,0.018638,0.019221,0.017911
3,r2,0.852187,0.003744,0.851531,0.851163,0.851859,0.848070,0.858314
4,residual_deviance,0.018580,0.000464,0.018565,0.018564,0.018638,0.019221,0.017911
5,rmse,0.136299,0.001705,0.136254,0.136250,0.136522,0.138639,0.133831
6,rmsle,0.017540,0.000183,0.017498,0.017515,0.017539,0.017830,0.017320



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2021-11-29 12:43:14,16 min 55.091 sec,0.0,0.354545,0.272643,0.125702
1,,2021-11-29 12:43:33,17 min 13.857 sec,5.0,0.244842,0.182514,0.059947
2,,2021-11-29 12:43:50,17 min 30.581 sec,10.0,0.191120,0.137140,0.036527
3,,2021-11-29 12:44:06,17 min 46.980 sec,15.0,0.160338,0.108873,0.025708
4,,2021-11-29 12:44:22,18 min 3.236 sec,20.0,0.145497,0.094778,0.021169
5,,2021-11-29 12:44:38,18 min 18.756 sec,25.0,0.137959,0.087702,0.019033
6,,2021-11-29 12:44:53,18 min 34.185 sec,30.0,0.133242,0.083359,0.017753
7,,2021-11-29 12:45:08,18 min 49.279 sec,35.0,0.130351,0.080857,0.016991
8,,2021-11-29 12:45:23,19 min 4.045 sec,40.0,0.128400,0.079242,0.016487
9,,2021-11-29 12:45:37,19 min 17.988 sec,45.0,0.127158,0.078292,0.016169



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,建築年,131377.953125,1.000000,0.388887
1,面積（㎡）,53113.847656,0.404283,0.157221
2,面積（㎡）容積率（％）_combi,30180.675781,0.229724,0.089337
3,都道府県名_te,28039.650391,0.213427,0.082999
4,市区町村名,23677.646484,0.180225,0.070087
5,都道府県名,6769.954590,0.051530,0.020040
6,agg_mean_最寄駅：距離（分）_grpby_市区町村名,5822.710938,0.044320,0.017236
7,最寄駅：距離（分）,5156.896484,0.039252,0.015265
8,改装,4231.681641,0.032210,0.012526
9,agg_count_最寄駅：距離（分）_grpby_市区町村名,3489.061279,0.026557,0.010328



See the whole table with table.as_data_frame()


In [43]:
# 予測
y_pred = aml.leader.predict(h2o_test).as_data_frame()
y_pred.index = test_df.index
y_pred

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


,predict
677392,7.265606
677393,7.290009
677394,7.255915
677395,7.176235
677396,6.954551
...,...
700538,7.100142
700539,7.623711
700540,7.341437
700541,7.483755


In [44]:
test_df[TARGET] = y_pred
test_df

,ID,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名_te,取引時点_何年前,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等,面積（㎡）容積率（％）_combi,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名
677392,14068534,11.0,60,1996.0,60.0,200.0,7.265606,61,7.268224,1,1,251,5181,1999,2,1,-1,0,0,1,-1,120.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0
677393,14068533,24.0,70,1998.0,50.0,100.0,7.290009,61,7.268224,1,1,251,5181,1999,1,0,0,0,9,0,-1,70.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0
677394,14068532,11.0,60,1996.0,60.0,200.0,7.255915,61,7.268224,1,1,251,5181,1999,1,1,-1,0,0,1,-1,120.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0
677395,14068420,21.0,105,1969.0,60.0,200.0,7.176235,61,7.268224,1,1,251,5018,1702,19,0,0,0,1,0,-1,210.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0
677396,14068219,21.0,65,1979.0,60.0,200.0,6.954551,61,7.268224,1,1,251,4480,1544,1,0,-1,0,7,1,-1,130.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700538,23000726,6.0,60,1974.0,60.0,200.0,7.100142,62,7.124361,0,4,88,1834,420,0,0,0,-1,1,0,-1,120.0,2133,2254,2250,2250,8.068917,69.134871,63.533333,249.480000,0.0,15,30.0,80.0,45.0,220,80.0,600.0
700539,23000787,11.0,80,2015.0,60.0,200.0,7.623711,62,7.124361,0,4,88,1887,465,1,0,0,0,8,1,-1,160.0,2133,2254,2250,2250,8.068917,69.134871,63.533333,249.480000,0.0,15,30.0,80.0,45.0,220,80.0,600.0
700540,13014303,8.0,20,2003.0,80.0,400.0,7.341437,62,7.396885,0,2,31,862,496,3,0,-1,0,4,1,-1,80.0,10397,11087,10969,10969,5.183995,37.694597,72.351171,446.294102,0.0,10,50.0,100.0,75.0,1000,80.0,900.0
700541,23001017,15.0,70,2014.0,60.0,200.0,7.483755,62,7.124361,0,4,88,551,733,1,0,0,0,7,1,-1,140.0,2133,2254,2250,2250,8.068917,69.134871,63.533333,249.480000,0.0,15,30.0,80.0,45.0,220,80.0,600.0


In [45]:
sub_df = pd.merge(sub_df[['ID']], test_df[['ID', TARGET]], on='ID')
sub_df.to_csv(dir_for_output + '/test_submission.csv', index=False)